In [1]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [5]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [6]:
# 오늘 날짜
today = datetime.today()#.strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제, 오늘이 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)
if today.weekday() == 5:  # 토요일
    today -= timedelta(days=1)
elif today.weekday() == 6:  # 일요일
    today -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')
today = today.strftime('%Y%m%d')

#### 법정동코드

In [321]:
dist_code = pd.read_csv('C:\py_src\midoproject\data/법정동코드 전체자료.csv',encoding='cp949')

In [322]:
dist_code_list = []
for i in range(dist_code['법정동명'].str.split().apply(len).max()):
    dist_code_temp = dist_code[dist_code['법정동명'].str.split().str[i].str[-1].isin(['읍','면','동','리','가','로']).dropna()]
    dist_code_list.append([dist_code_temp['법정동명'].str.split().str[i].dropna().unique()])

emd_code_list = list(chain(*chain(*dist_code_list)))

#### 종합쇼핑몰 납품상세내역

In [323]:
all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_all', bigquerykey_path)

In [324]:
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '수요기관코드', '수요기관구분', '수요기관지역명','납품요구지청명']]
all_shop_df_fin = all_shop_df_fin.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [325]:
# 특수문자, 숫자, 영어 제거 함수 (연속된 공백을 단일 공백으로 변환 포함)
def clean_text(text):
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', ' ', text)
    text = re.sub('\s+', ' ', text).strip()  # 연속된 공백을 단일 공백으로 변환
    return text

In [326]:
# 사업명 전처리
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명'].apply(clean_text)
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.strip()

all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].apply((' ').join)

In [327]:
# 수요기관 필터링
dist_nm = pd.Series(all_shop_df_fin['수요기관지역명'].str.split().str[1].dropna().unique())
dist_nm = np.where(dist_nm.apply(len)<=2,dist_nm,dist_nm.str[:-1])

In [14]:
# 종합쇼핑몰 학교 관련 사업
need_edu_kwd = '학교|초|초등학교|중|중학교|고|고등학교|대|대학교|교육|교육청|교육지원청|교육부|공립|사립|국립'
shop_edu_mapping_temp1 = all_shop_df_fin[all_shop_df_fin['납품요구건명'].str.contains(need_edu_kwd)]
shop_edu_mapping_temp2 = all_shop_df_fin[all_shop_df_fin['수요기관명'].str.contains(need_edu_kwd)]
shop_edu_mapping_df = pd.concat([shop_edu_mapping_temp1,shop_edu_mapping_temp2],axis=0).drop_duplicates().reset_index(drop=True)
shop_edu_mapping_df

,납품요구접수일자,수요기관명,납품요구건명,업체명,단가,단위,수량,금액,수요기관코드,수요기관구분,수요기관지역명,납품요구지청명,납품요구건명_re
0,2024-07-10,광주광역시교육청 광주석산고등학교,광주석산고 관급자재(인조잔디) 구매,주식회사 유니스포텍,41200,㎡,1402.0,57762400,7380054,기타기관,광주광역시 남구,광주지방조달청,광주석산고 관급자재 인조잔디 구매
1,2024-07-10,한국중부발전(주)신서천발전본부,춘장대사택 테니스장 인조잔디 구매,주식회사 네오필드,36300,㎡,0.0,0,Z000438,공기업,충청남도 서천군,대전지방조달청,춘장대사택 테니스장 인조잔디 구매
2,2024-07-10,대전광역시동부교육청 동명중학교,동명중학교 운동장 환경개선 조성 및 기타공사 인조잔디 구매,주식회사 베노,46500,㎡,1097.0,51010500,7440035,기타기관,대전광역시 중구,대전지방조달청,동명중학교 운동장 환경개선 조성 기타공사 인조잔디 구매
3,2024-07-09,서울특별시강서교육청 서울양명초등학교,양명초 인조잔디 구입,주식회사 이원,63800,㎡,356.0,22712800,7080084,교육기관,서울특별시 양천구,서울지방조달청,양명초 인조잔디 구입
4,2024-07-09,대구광역시남부교육청 성지중학교,성지중학교 운동장 인조잔디( t55mm) 구입,주식회사 플랜에이,63200,㎡,2727.0,172346400,7270193,교육기관,대구광역시 달서구,대구지방조달청,성지중학교 운동장 인조잔디 구입
...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,2024-01-12,부산광역시교육청 한국조형예술고등학교,인조잔디공사 관급자재 인조잔디 구입,주식회사 네오필드,61500,㎡,1654.0,101721000,7150579,교육기관,부산광역시 남구,부산지방조달청,인조잔디공사 관급자재 인조잔디 구입
410,2024-01-10,경기도교육청 경기도김포교육청 마송중학교,간이체육활동공간조성공사 관급자재(인조잔디) 구입,주식회사 유니스포텍,62900,㎡,269.0,16920100,7770121,교육기관,경기도 김포시,인천지방조달청,간이체육활동공간조성공사 관급자재 인조잔디 구입
411,2024-01-09,서울특별시강동교육청 서울가동초등학교,실내운동장 인조잔디 구입,주식회사 필드글로벌,63000,㎡,495.0,31185000,7070052,교육기관,서울특별시 송파구,서울지방조달청,실내운동장 인조잔디 구입
412,2024-01-05,대전광역시,외삼족구장 시설개선사업 관급자재,주식회사 미도플러스 (MIDOPLUS Inc.),53000,㎡,3721.0,197213000,6300000,지자체,대전광역시 서구,대전지방조달청,외삼족구장 시설개선사업 관급자재


In [15]:
# 교육청, 교육지원청, 학교명 필터링
shop_edu_mapping_df['수요기관명_re'] = shop_edu_mapping_df['수요기관명'].str.replace('교육청','').str.replace('교육지원청','')
shop_edu_kwd = set(chain(*shop_edu_mapping_df['수요기관명_re'].str.split()))

In [16]:
# 학교 관련 키워드 데이터셋
edu_mapping_keywd_all = list(chain(*shop_edu_mapping_df['납품요구건명_re'].str.split(' '))) ## 전체 split 키워드
edu_mapping_keywd_df = pd.DataFrame(Counter(edu_mapping_keywd_all).items(),columns=['키워드','빈도수'])
edu_mapping_keywd_df = edu_mapping_keywd_df[edu_mapping_keywd_df['키워드'].apply(len)!=1].reset_index(drop=True)
edu_mapping_keywd_df.sort_values('빈도수',ascending=False)

,키워드,빈도수
2,인조잔디,396
1,관급자재,217
12,구입,139
7,운동장,100
3,구매,93
...,...,...
271,소프트볼연습장,1
270,서울신정고,1
269,고덕생활축구장,1
266,고흥동초,1


#### 교육청 예산서

In [595]:
# 빅쿼리에서 불러오기
edu_budget_df = get_dataframe_from_bigquery('edu','edu_budget',bigquerykey_path)
edu_budget_df.sort_values('시도').reset_index(drop=True)

,시도,시군구,구분,과업명,금액,면적,예산집행
0,강원특별자치도,화천교육지원청,공립,상서중학교 운동장 보수,"89,533,000","4,664",학교
1,강원특별자치도,인제교육지원청,공립,상남중학교 개방형 운동장 조성,"725,302,000","6,507",교육청
2,강원특별자치도,강원특별자치도교육청,공립,강릉제일고등학교 축구장 인조잔디 교체,"1,020,560,000","8,576",교육청
3,강원특별자치도,강릉교육지원청,공립,관동중학교 인조잔디 운동장 조성,"806,400,000","4,082",교육청
4,강원특별자치도,강원특별자치도교육청,사립,동해광희고등학교 인조잔디운동장 보수,"51,700,000",None,None
...,...,...,...,...,...,...,...
189,충청남도,보령교육지원청,공립,성주초등학교 운동장 교체,"70,980,000",None,None
190,충청남도,당진교육지원청,공립,면천중학교 운동장 교체,"297,561,000",None,None
191,충청북도,충청북도교육청,공립,충북산업과학고등학교 다목적구장 조성,"769,799,000","5,540",교육청
192,충청북도,청주교육지원청,공립,서경중학교 운동장 보수 및 바닥교체,"602,871,000","3,500",교육청


In [596]:
# 전처리
edu_budget_df['과업명_re'] = edu_budget_df['과업명'].apply(clean_text)
# edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.strip()

In [597]:
# 학교 관련 키워드 및 데이터 추출
edu_budget_df['키워드'] = edu_budget_df['과업명'].apply(lambda x: ''.join(re.findall('\\w+초등학교|\\w+중학교|\\w+고등학교|\\w+학교', x)))
edu_budget_df_filter = edu_budget_df[edu_budget_df['키워드']!=''].reset_index(drop=True)

In [603]:
edu_budget_df_filter['키워드'].str.split('초등학교|중학교|고등학교|학교').str[0].unique()

array(['', '세화', '한라', '북평여자', '부안', '봉의', '춘천', '무주', '이리동남', '전주생명과학',
       '무안', '광주수창', '광주북성', '광주제일', '성주', '청파', '면천', '합덕제철', '재송', '문현',
       '사직', '향동', '흥동', '광명북', '매양', '부양', '오남', '군포', '신양', '회천', '보산',
       '부천중원', '수성', '시흥능곡', '신천', '선부', '비룡', '문원', '호계', '세종', '백학',
       '어정', '서원', '의정부', '장호원', '와석', '현화', '영북', '기산', '매홀', '정목', '상봉',
       '덕산', '삼각산', '계양', '제물포', '덕적', '삼산', '경포', '강릉', '상서', '공립', '송지',
       '문향', '신지', '목포중앙여', '지세포', '대운', '문산', '진주', '도계', '봉림', '양덕',
       '창원상남', '통영', '하동', '대구공업', '성지', '돌마', '선일', '정지', '점봉', '연천노곡',
       '백봉', '역북', '대월', '해솔', '강화', '부평', '관동', '강릉제일', '상남', '대전동신과학',
       '군산문화', '간중', '용진', '삼기', '전일', '한들', '순창제일', '병영', '석교', '화순제일',
       '장흥남', '서경', '강경여자', '부여여자', '예산여자', '충북산업과학', '계림', '운수', '구미인덕',
       '형곡', '선산', '구미', '강구', '고경', '지보', '예천동부', '예천', '비안', '단밀', '대보',
       '상대', '포항항도', '칠천', '월천', '이작', '금서', '오부', '김해내동', '군북', '청덕',
       '경북', '대구', '온양', '옥동', '강원대', '원주삼육', '

In [533]:
edu_kwd_list = '|'.join(edu_budget_df['키워드'].str.split('초등학교|중학교|고등학교|학교').str[0].str.strip().dropna().unique())
edu_kwd_list

'완주 스포츠클럽 실외야구장|친환경 운동장 조성사업비 지원 교|운동장 개보수 고|친환경운동장조성 다목적구장|교육환경개선 기타 사업 개교|군산청소년자치배움터 자몽 인조잔디|완주 스포츠클럽 실내야구장|완주 스포츠클럽 실외 야구장 설계|운동장 조성 초 중|운동장개보수 다시초|세화|한라|북평여자|부안|봉의|춘천|무주|이리동남|전주생명과학|무안|인조잔디운동장보수 및 유지관리 초 고 공립 교|광주수창|광주북성|광주제일|성주|청파|면천|합덕제철|재송|문현|사직|향동|흥동|광명북|매양|부양|오남|군포|신양|회천|보산|부천중원|수성|시흥능곡|신천|선부|비룡|문원|호계|세종|백학|어정|서원|의정부|장호원|와석|현화|영북|기산|매홀|정목|상봉|덕산|삼각산||계양|제물포|덕적|삼산|경포|강릉|상서|공립|송지|문향|신지|목포중앙여|지세포|대운|문산|진주|도계|봉림|양덕|창원상남|통영|하동|대구공업|성지|돌마|선일|정지|점봉|연천노곡|백봉|역북|대월|해솔|강화|부평|관동|강릉제일|상남|대전동신과학|군산문화|간중|용진|삼기|전일|한들|순창제일|운동장 개보수 공립|병영|석교|화순제일|장흥남|서경|강경여자|부여여자|예산여자|충북산업과학|계림|운수|구미인덕|형곡|선산|구미|강구|고경|지보|예천동부|예천|비안|단밀|대보|상대|포항항도|칠천|월천|이작|미리벌중 양궁장 신축 계속비|금서|오부|김해내동|군북|청덕|경북|대구|온양|옥동|강원대|원주삼육|동해광희|유일여자|인조잔디운동장보수 및 유지관리 중 고 사립 교|광주석산|신평|건국|혜광|동의|배정|부산동암|해원|여주제일|라온|사립|덕인|수륜|영일|정곡|대륜|익산|다양한'

In [547]:
all_shop_df_fin['납품요구건명'].apply(lambda x: ''.join(re.findall('\\w+초등학교|\\w+중학교|\\w+고등학교|\\w+학교', x))).unique()

array(['', '동명중학교', '성지중학교', '재송중학교', '운산고등학교', '장승중학교', '문현초등학교',
       '서울영신초등학교', '상산고등학교', '목포정명여자고등학교', '오산대학교', '선린대학교', '상봉중학교',
       '인성초등학교', '용동중학교', '충남삼성고등학교', '경북고등학교', '용곡중학교', '이천중학교',
       '대신고등학교', '광희중학교', '삼각산중학교', '원주삼육중학교', '포항제철고등학교', '대전외국인학교',
       '신림중학교', '미래학교', '공릉중학교', '서울양진초등학교', '서울동산고등학교', '줄포자동차공업고등학교',
       '삽교고등학교', '성동고등학교', '연희중학교', '지세포중학교', '진주중학교', '태안중학교', '덕산중학교',
       '야로중학교', '광양중학교', '청구고등학교', '신명중학교', '서울봉래초등학교', '전북대학교',
       '강원외국어고등학교', '효암고등학교미래학교', '서울금북초등학교', '천안초등학교', '배명고등학교', '김제중학교',
       '동명대학교', '대명중학교', '광석초등학교', '고등학교', '그린스마트미래학교', '금성초등학교',
       '관인고등학교', '청도고등학교', '율면고등학교', '중앙기독중학교', '신례원초등학교', '순천조례초등학교',
       '국립공주대학교', '경북체육중고등학교', '기업고등학교', '운천중학교', '고창북중학교', '목포영산초등학교',
       '중고등학교', '목포부주초등학교', '화산초등학교', '용호고등학교', '대전제일고등학교', '서울디지텍고등학교',
       '신설대체이전학교', '목포유달초등학교', '부산개성중학교', '철성고등학교', '소화초등학교', '불은초등학교',
       '정남중학교', '세경고등학교', '만경중학교', '배정미래고등학교', '건국고등학교', '원곡중학교'],
      dtype=object)

In [330]:
# 지역명, 교육청명 종합쇼핑몰 형식과 통일
# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
edu_budget_df['시도_re'] = edu_budget_df['시도'].replace(replacements)
edu_budget_df['시도_re'] = edu_budget_df['시도_re'].str[:2]

edu_budget_df['시군구_re'] = np.where(edu_budget_df['시군구'].str.contains('교육지원청'),
                                   edu_budget_df['시도'] + edu_budget_df['시군구'].str.replace('교육지원청',''), edu_budget_df['시군구']) 

In [20]:
# # 교육청, 교육지원청, 학교명 기준 추출
# filtered_edu_budget_df = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(shop_edu_kwd)],
#                                     edu_budget_df[edu_budget_df['과업명'].str.contains('|'.join(shop_edu_kwd))],
#                                     edu_budget_df[edu_budget_df['과업명_re'].str.contains('|'.join(shop_edu_kwd))]],axis=0).drop_duplicates().reset_index(drop=True)

#### 교육청 종합쇼핑몰 전처리

In [331]:
# Initialize the dictionary for split keywords
shop_split_kwd = {}
for i in range(all_shop_df_fin['납품요구건명_re'].str.split().apply(len).max()):
    shop_split_kwd['kwd_' + str(i)] = all_shop_df_fin['납품요구건명_re'].str.split().str[i]

In [332]:
# 매핑데이터셋
mapping_merge_df = []

In [333]:
# 종합쇼핑몰 납품요구건명 split 키워드 순서대로 맵핑
# Initialize the list to store the results
prep_edu_df_today = edu_budget_df.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

# Iterate over each keyword column
for i in range(len(shop_split_kwd)):
    keyword_column = 'kwd_' + str(i)
    
    # Get unique non-null keywords
    shop_split_kwd_temp = shop_split_kwd[keyword_column].dropna().unique()

    prep_edu_df_today[keyword_column] = prep_edu_df_today['과업명_re'].apply(lambda x: ' '.join(re.findall('|'.join(shop_split_kwd_temp), x)))
    prep_edu_df_today[keyword_column] = prep_edu_df_today[keyword_column].str.strip()
    prep_edu_df_today[keyword_column] = prep_edu_df_today[keyword_column].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall('|'.join(shop_split_kwd_temp), x)))
    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin[keyword_column].str.strip()
    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin[keyword_column].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

    mapping_merge_df_temp = pd.merge(
            prep_edu_df_today[(prep_edu_df_today[keyword_column] != '') & ~(prep_edu_df_today[keyword_column].isnull())][['시도','시군구','과업명', '과업명_re','시도_re','시군구_re', keyword_column]].reset_index(drop=True),
            prep_all_shop_df_fin[(prep_all_shop_df_fin[keyword_column] != '') & ~(prep_all_shop_df_fin[keyword_column].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', keyword_column]].reset_index(drop=True),
            how='left',
            on=keyword_column
            ).drop_duplicates().reset_index(drop=True)
    
    # Append the temporary DataFrame to the results list
    mapping_merge_df.append(mapping_merge_df_temp)

In [335]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 추출
# Initialize the columns for district keywords
prep_edu_df_today = edu_budget_df.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

sig_pat = '|'.join(dist_nm)

prep_edu_df_today['kwd_sig'] = prep_edu_df_today['과업명_re'].apply(lambda x: ' '.join(re.findall(sig_pat, x)))
prep_edu_df_today['kwd_sig'] = prep_edu_df_today['kwd_sig'].str.strip()
prep_edu_df_today['kwd_sig'] = prep_edu_df_today['kwd_sig'].str.split()
prep_edu_df_today = prep_edu_df_today.explode('kwd_sig')

prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(sig_pat, x)))
prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['kwd_sig'].str.strip()
prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['kwd_sig'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_sig')

mapping_merge_df_sig = pd.merge(
    prep_edu_df_today[(prep_edu_df_today['kwd_sig'] != '') & ~(prep_edu_df_today['kwd_sig'].isnull())][['시도','시군구','과업명', '과업명_re','시도_re','시군구_re', 'kwd_sig']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_sig'] != '') & ~(prep_all_shop_df_fin['kwd_sig'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_sig']].reset_index(drop=True),
    how='left',
    on='kwd_sig'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_sig)

In [338]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 읍면동 추출
# Initialize the columns for district keywords
prep_edu_df_today = edu_budget_df.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

emd_pat = '|'.join(set(emd_code_list))

prep_edu_df_today['kwd_emd'] = prep_edu_df_today['과업명_re'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
prep_edu_df_today['kwd_emd'] = prep_edu_df_today['kwd_emd'].str.strip()
prep_edu_df_today['kwd_emd'] = prep_edu_df_today['kwd_emd'].str.split()
prep_edu_df_today = prep_edu_df_today.explode('kwd_emd')

prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['kwd_emd'].str.strip()
prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['kwd_emd'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_emd')

mapping_merge_df_emd = pd.merge(
    prep_edu_df_today[(prep_edu_df_today['kwd_emd'] != '') & ~(prep_edu_df_today['kwd_emd'].isnull())][['시도','시군구','과업명', '과업명_re','시도_re','시군구_re', 'kwd_emd']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_emd'] != '') & ~(prep_all_shop_df_fin['kwd_emd'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_emd']].reset_index(drop=True),
    how='left',
    on='kwd_emd'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_emd)

In [346]:
# 필요키워드 앞부분 추출
need_edu_kwd = '학교|초|초등학교|중|중학교|고|고등학교|대|대학교|교육|교육청|교육지원청|교육부|공립|사립|국립'
need_kwd_split = list(set(chain(*all_shop_df_fin['납품요구건명_re'].apply(lambda x: re.split(need_edu_kwd,x)).apply(''.join).str.split())))
need_kwd_split_list = list(pd.Series(need_kwd_split)[pd.Series(need_kwd_split).apply(len) >= 2])

In [348]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 앞부분 매핑
# Initialize the columns for district keywords
prep_edu_df_today = edu_budget_df.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

kwd_split_pat = '|'.join(set(need_kwd_split_list))

prep_edu_df_today['kwd_need_split'] = prep_edu_df_today['과업명_re'].apply(lambda x: ' '.join(re.findall(kwd_split_pat, x)))
prep_edu_df_today['kwd_need_split'] = prep_edu_df_today['kwd_need_split'].str.strip()
prep_edu_df_today['kwd_need_split'] = prep_edu_df_today['kwd_need_split'].str.split()
prep_edu_df_today = prep_edu_df_today.explode('kwd_need_split')

prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(kwd_split_pat, x)))
prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['kwd_need_split'].str.strip()
prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['kwd_need_split'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_need_split')

mapping_merge_df_need_split = pd.merge(
    prep_edu_df_today[(prep_edu_df_today['kwd_need_split'] != '') & ~(prep_edu_df_today['kwd_need_split'].isnull())][['시도','시군구','과업명', '과업명_re','시도_re','시군구_re', 'kwd_need_split']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_need_split'] != '') & ~(prep_all_shop_df_fin['kwd_need_split'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need_split']].reset_index(drop=True),
    how='left',
    on='kwd_need_split'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need_split)

In [351]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 매핑
# Initialize the columns for district keywords
prep_edu_df_today = edu_budget_df.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

prep_edu_df_today['kwd_need'] = prep_edu_df_today['과업명_re'].apply(lambda x: ' '.join(re.findall(need_edu_kwd, x)))
prep_edu_df_today['kwd_need'] = prep_edu_df_today['kwd_need'].str.strip()
prep_edu_df_today['kwd_need'] = prep_edu_df_today['kwd_need'].str.split()
prep_edu_df_today = prep_edu_df_today.explode('kwd_need')

prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(need_edu_kwd, x)))
prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['kwd_need'].str.strip()
prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['kwd_need'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_need')

mapping_merge_df_need = pd.merge(
    prep_edu_df_today[(prep_edu_df_today['kwd_need'] != '') & ~(prep_edu_df_today['kwd_need'].isnull())][['시도','시군구','과업명', '과업명_re','시도_re','시군구_re', 'kwd_need']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_need'] != '') & ~(prep_all_shop_df_fin['kwd_need'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need']].reset_index(drop=True),
    how='left',
    on='kwd_need'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need)

In [353]:
# 키워드별 데이터셋 병합
mapping_merge_df_fin = pd.concat(mapping_merge_df, axis=0).drop_duplicates()
mapping_merge_df_fin = mapping_merge_df_fin[~mapping_merge_df_fin['납품요구건명'].isnull()].reset_index(drop=True)

In [355]:
# 모든 키워드 합
kwd_columns = ['kwd_0','kwd_1','kwd_2','kwd_3','kwd_4','kwd_5','kwd_6','kwd_7','kwd_8','kwd_sig','kwd_emd','kwd_need_split','kwd_need']
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin[kwd_columns].apply(lambda x : (' ').join(x.dropna()),axis=1)
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin['all_kwd'].str.strip()
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin['all_kwd'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_fin.drop(columns=kwd_columns, inplace=True)

In [357]:
# 지역명 추출1
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[0]

# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].replace(replacements)
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].str[:2]

In [359]:
# 지역명 추출2
mapping_merge_df_fin['수요기관지역명_split2'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[1]

mapping_merge_df_fin['수요기관지역명_split2'] = np.where(mapping_merge_df_fin['수요기관지역명_split2'].isnull(),
                                                  mapping_merge_df_fin['수요기관지역명_split1'],
                                                  mapping_merge_df_fin['수요기관지역명_split2'])

mapping_merge_df_fin['수요기관지역명_split2'] = np.where(mapping_merge_df_fin['수요기관지역명_split2'].apply(len)<=2,
                                                  mapping_merge_df_fin['수요기관지역명_split2'],
                                                  mapping_merge_df_fin['수요기관지역명_split2'].str[:-1])

dist_sig_pat = '|'.join(mapping_merge_df_fin['수요기관지역명_split2'].dropna().unique())

mapping_merge_df_fin['시군구_re2'] = mapping_merge_df_fin['시군구_re'].apply(lambda x: ' '.join(re.findall(dist_sig_pat, x)))
mapping_merge_df_fin['시군구_re2'] = mapping_merge_df_fin['시군구_re2'].str.strip()
mapping_merge_df_fin['시군구_re2'] = mapping_merge_df_fin['시군구_re2'].str.split()
mapping_merge_df_fin = mapping_merge_df_fin.explode('시군구_re2')

In [366]:
# 제외 지역 제거
filtered_mapping_merge_df_fin = mapping_merge_df_fin[mapping_merge_df_fin['시군구_re2']!=''].drop_duplicates().reset_index(drop=True)
filtered_mapping_merge_df_fin = filtered_mapping_merge_df_fin.fillna('')

In [369]:
# 지역 일치 확인1
mapping_merge_df_fin_temp1 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['시도_re']==filtered_mapping_merge_df_fin['수요기관지역명_split1']].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['시군구_re2'].str.contains('|'.join(mapping_merge_df_fin_temp1['수요기관지역명_split2'].dropna().unique()))].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['시군구_re2']==mapping_merge_df_fin_temp1['수요기관지역명_split2']].drop_duplicates().reset_index(drop=True)

# 본청 포함 확인
mapping_merge_df_fin_temp2 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['시군구'].str.contains('본청')].drop_duplicates().reset_index(drop=True)

# 데이터 병합
mapping_merge_df_final = pd.concat([mapping_merge_df_fin_temp1,mapping_merge_df_fin_temp2], axis=0).drop_duplicates().reset_index(drop=True)

In [449]:
# 그룹화할 컬럼 리스트
group_cols = ['시도', '시군구', '과업명', '과업명_re', '시도_re', '시군구_re', '수요기관명', '수요기관지역명','납품요구건명', '납품요구건명_re', '수요기관지역명_split1', '수요기관지역명_split2', '시군구_re2']

# 그룹화 및 all_kwd 컬럼 조인
mapping_merge_df_final_grp = mapping_merge_df_final.groupby(group_cols).agg({'all_kwd': ' '.join}).reset_index()
mapping_merge_df_final_grp['all_kwd'] = mapping_merge_df_final_grp['all_kwd'].str.strip()
mapping_merge_df_final_grp['all_kwd'] = mapping_merge_df_final_grp['all_kwd'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [450]:
# 필요키워드 기준
mapping_merge_df_final_grp['과업명_키워드1'] = mapping_merge_df_final_grp['과업명'].apply(lambda x: ' '.join(re.findall(need_edu_kwd, x)))
mapping_merge_df_final_grp['납품요구건명_키워드1'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(need_edu_kwd, x)))

# # 사업명 지역 추출
# mapping_merge_df_final_grp['과업명_지역'] = mapping_merge_df_final_grp['과업명'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
# mapping_merge_df_final_grp['납품요구건명_지역'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))

# 필요키워드 패턴 기준
need_kwd_pat = '\w+' + ('|\w+').join(need_edu_kwd.split('|'))

mapping_merge_df_final_grp['과업명_키워드2'] = mapping_merge_df_final_grp['과업명'].apply(lambda x: ' '.join(re.findall(need_kwd_pat, x)))
mapping_merge_df_final_grp['납품요구건명_키워드2'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(need_kwd_pat, x)))

mapping_merge_df_final_grp['과업명_키워드2'] = mapping_merge_df_final_grp['과업명_키워드2'].replace({'초등학교':'초','중학교':'중','고등학교':'고','대학교':'대'}, regex=True)
mapping_merge_df_final_grp['납품요구건명_키워드2'] = mapping_merge_df_final_grp['납품요구건명_키워드2'].replace({'초등학교':'초','중학교':'중','고등학교':'고','대학교':'대'}, regex=True)

In [451]:
# mapping_merge_df_final_grp.to_csv('C:\py_src\midoproject\data/edu_mapping.csv',encoding='utf-8-sig',index=False)

In [452]:
def check_inclusion1(row):
    set1 = set(row['과업명_키워드1'])
    set2 = set(row['납품요구건명_키워드1'])
    return bool(set1 & set2)

In [453]:
# 필요키워드 모두 없는것 제외
mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['과업명_키워드1']!='')&(mapping_merge_df_final_grp['납품요구건명_키워드1']!='')].reset_index(drop=True)

# 필요키워드 포함여부 확인
mapping_merge_df_final_grp_temp1['과업명_키워드1'] = mapping_merge_df_final_grp_temp1['과업명_키워드1'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'] = mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp_temp1[mapping_merge_df_final_grp_temp1.apply(check_inclusion1,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp1['과업명_키워드1'] = mapping_merge_df_final_grp_temp1['과업명_키워드1'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'] = mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

# # 
# mapping_merge_df_final_grp_temp1['과업명_키워드1'] = mapping_merge_df_final_grp_temp1['과업명_키워드1'].str.split()
# mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp_temp1.explode('과업명_키워드1').drop_duplicates().reset_index(drop=True)

# mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'] = mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'].str.split()
# mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp_temp1.explode('납품요구건명_키워드1').drop_duplicates().reset_index(drop=True)

In [260]:
def check_inclusion2(row):
    set1 = set(row['과업명_키워드2'])
    set2 = set(row['납품요구건명_키워드2'])
    return bool(set1 & set2)

In [261]:
# 필요키워드패턴 모두 없는것 제외
mapping_merge_df_final_grp_temp2 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['과업명_키워드2']!='')&(mapping_merge_df_final_grp['납품요구건명_키워드2']!='')].reset_index(drop=True)

# 필요키워드 패턴 포함여부 확인
mapping_merge_df_final_grp_temp2['과업명_키워드2'] = mapping_merge_df_final_grp_temp2['과업명_키워드2'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_temp2['납품요구건명_키워드2'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_temp2 = mapping_merge_df_final_grp_temp2[mapping_merge_df_final_grp_temp2.apply(check_inclusion2,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp2['과업명_키워드2'] = mapping_merge_df_final_grp_temp2['과업명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_temp2['납품요구건명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [312]:
# 공통된 글자를 찾는 함수 정의
def find_common_chars_in_order(str1, str2):
    common_chars = [char for char in str1 if char in str2]
    return ''.join(common_chars)

In [313]:
# 필요키워드패턴 모두 없는것 제외
mapping_merge_df_final_grp_temp3 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['과업명_키워드2']!='')&(mapping_merge_df_final_grp['납품요구건명_키워드2']!='')].reset_index(drop=True)


# apply 함수를 사용하여 공통된 글자 찾기
mapping_merge_df_final_grp_temp3['공통키워드'] = mapping_merge_df_final_grp_temp3.apply(lambda row: find_common_chars_in_order(row['과업명_키워드2'], row['납품요구건명_키워드2']), axis=1)

In [320]:
mapping_merge_df_final_grp_temp3[mapping_merge_df_final_grp_temp3['공통키워드'].apply(len) > 1]

,시도,시군구,과업명,과업명_re,시도_re,시군구_re,수요기관명,수요기관지역명,납품요구건명,납품요구건명_re,수요기관지역명_split1,수요기관지역명_split2,시군구_re2,all_kwd,과업명_키워드1,납품요구건명_키워드1,과업명_키워드2,납품요구건명_키워드2,공통키워드
2,강원특별자치도,원주교육지원청,원주삼육중학교 소운동장 인조잔디 개보수,원주삼육중학교 소운동장 인조잔디 개보수,강원,강원특별자치도원주,강원특별자치도교육청 강원특별자치도원주교육지원청 원주삼육중학교,강원도 원주시,원주삼육중학교 풋살장 개보수 인조잔디 구입,원주삼육중학교 풋살장 개보수 인조잔디 구입,강원,원주,원주,개보수 원주 원주삼육 인조잔디 중 학교,중 학교,중 학교,원주삼육중,원주삼육중,원주삼육중
15,경기도,수원교육지원청,수성중학교 친환경 운동장 조성 사업,수성중학교 친환경 운동장 조성 사업,경기,경기도수원,경기도수원교육청 수원북중학교,경기도 수원시 장안구,수원북중 인조잔디(띠잔디) 구입,수원북중 인조잔디 띠잔디 구입,경기,수원,수원,중,중 학교,중,수성중,수원북중,수중
18,경기도,연천교육지원청,연천노곡초등학교 체육관 증축,연천노곡초등학교 체육관 증축,경기,경기도연천,경기도연천교육청 연천초등학교,경기도 연천군,연천초 인조잔디 구매,연천초 인조잔디 구매,경기,연천,연천,연천 초,초 학교,초,연천노곡초,연천초,연천초
51,경상남도,거제교육지원청,지세포중학교 인조잔디 철거,지세포중학교 인조잔디 철거,경남,경상남도거제,경상남도거제교육청 지세포중학교,경상남도 거제시,지세포중학교 인조잔디 재조성 및 트랙 개선공사 관급자재(인조잔디) 구입,지세포중학교 인조잔디 재조성 트랙 개선공사 관급자재 인조잔디 구입,경남,거제,거제,인조잔디 중 지세포 학교,중 학교,중 학교,지세포중,지세포중,지세포중
56,경상남도,진주교육지원청,진주중학교 인조잔디 교체,진주중학교 인조잔디 교체,경남,경상남도진주,경상남도진주교육청 진주중학교,경상남도 진주시,진주중학교 운동장 인조잔디 교체공사 관급자설치관급자재(인조잔디) 구입,진주중학교 운동장 인조잔디 교체공사 관급자설치관급자재 인조잔디 구입,경남,진주,진주,인조잔디 중 진주 학교,중 학교,중 학교,진주중,진주중,진주중
59,경상북도,영덕교육지원청,강구중학교(축구부 쉼터 증축),강구중학교 축구부 쉼터 증축,경북,경상북도영덕,경상북도 영덕군 시설체육사업소,경상북도 영덕군,강구대게축구장 관중석 하단부 철거공사 관급자재 인조잔디 구입,강구대게축구장 관중석 하단부 철거공사 관급자재 인조잔디 구입,경북,영덕,영덕,중,중 학교,대 중,강구중,강구대 관중,강구중
68,경상북도,포항교육지원청,포항항도중학교(운동장 재조성),포항항도중학교 운동장 재조성,경북,경상북도포항,경상북도교육청 포항제철고등학교,경상북도 포항시 남구,포항제철고등학교 다목적 운동부 체육시설 관급자재 구매,포항제철고등학교 다목적 운동부 체육시설 관급자재 구매,경북,포항,포항,포항 학교,중 학교,고 학교,포항항도중,포항제철고,포항항
72,부산광역시,해운대교육지원청,재송중학교 인조잔디 (재) 조성,재송중학교 인조잔디 재 조성,부산,부산광역시해운대,부산광역시해운대교육청 재송중학교,부산광역시 해운대구,재송중학교 인조잔디 및 배수판 구입,재송중학교 인조잔디 배수판 구입,부산,해운대,해운대,인조잔디 재송 재송중학교 중 학교,중 학교,중 학교,재송중,재송중,재송중
74,서울특별시,강서양천교육지원청,정목초등학교(운동장 환경 개선),정목초등학교 운동장 환경 개선,서울,서울특별시강서양천,서울특별시강서교육청 서울정목초등학교,서울특별시 양천구,서울정목초 운동장환경개선 인조잔디 구매,서울정목초 운동장환경개선 인조잔디 구매,서울,양천,양천,운동장 초,초 학교,초,정목초,서울정목초,정목초
75,서울특별시,성북강북교육지원청,삼각산중학교(운동장 환경 개선),삼각산중학교 운동장 환경 개선,서울,서울특별시성북강북,서울특별시교육청 서울특별시성북강북교육지원청,서울특별시 강북구,관급자재[삼선초 외 1교(석관중) 운동장 환경개선공사 인조잔디] 구매,관급자재 삼선초 석관중 운동장 환경개선공사 인조잔디 구매,서울,강북,강북,운동장 중,중 학교,초 중,삼각산중,삼선초 석관중,삼중


In [117]:
# 필터링 데이터셋 병합
mapping_merge_df_final_grp_fin = pd.concat([mapping_merge_df_final_grp_temp1,mapping_merge_df_final_grp_temp2]).drop_duplicates().reset_index(drop=True)
mapping_merge_df_final_grp_fin

,시도,시군구,과업명,과업명_re,시도_re,시군구_re,수요기관명,수요기관지역명,납품요구건명,납품요구건명_re,수요기관지역명_split1,수요기관지역명_split2,시군구_re2,all_kwd,과업명_키워드1,납품요구건명_키워드1,과업명_키워드2,납품요구건명_키워드2
0,강원특별자치도,원주교육지원청,원주삼육중학교 소운동장 인조잔디 개보수,원주삼육중학교 소운동장 인조잔디 개보수,강원,강원특별자치도원주,강원특별자치도교육청 강원특별자치도원주교육지원청,강원특별자치도 원주시,(가칭)기업고등학교 신축공사 인조잔디 구입 및 설치,가칭 기업고등학교 신축공사 인조잔디 구입 설치,강원,원주,원주,인조잔디 학교,중 학교,고 학교,원주삼육중학교,기업고등학교
1,강원특별자치도,원주교육지원청,원주삼육중학교 소운동장 인조잔디 개보수,원주삼육중학교 소운동장 인조잔디 개보수,강원,강원특별자치도원주,강원특별자치도교육청 강원특별자치도원주교육지원청 원주삼육중학교,강원도 원주시,원주삼육중학교 풋살장 개보수 인조잔디 구입,원주삼육중학교 풋살장 개보수 인조잔디 구입,강원,원주,원주,개보수 원주 원주삼육 인조잔디 중 학교,중 학교,중 학교,원주삼육중학교,원주삼육중학교
2,강원특별자치도,춘천교육지원청,봉의초등학교 테니스장교체,봉의초등학교 테니스장교체,강원,강원특별자치도춘천,강원특별자치도교육청 강원특별자치도춘천교육지원청,강원도 춘천시,신동초 이전 신축공사 관급자재(인조잔디) 구입,신동초 이전 신축공사 관급자재 인조잔디 구입,강원,춘천,춘천,초,초 학교,초,봉의초등학교,신동초
3,강원특별자치도,춘천교육지원청,부안초등학교 인조잔디 운동장 및 우레탄조성,부안초등학교 인조잔디 운동장 및 우레탄조성,강원,강원특별자치도춘천,강원특별자치도교육청 강원특별자치도춘천교육지원청,강원도 춘천시,신동초 이전 신축공사 관급자재(인조잔디) 구입,신동초 이전 신축공사 관급자재 인조잔디 구입,강원,춘천,춘천,인조잔디 초,초 학교,초,부안초등학교,신동초
4,강원특별자치도,춘천교육지원청,춘천초등학교 다목적구장조성,춘천초등학교 다목적구장조성,강원,강원특별자치도춘천,강원특별자치도교육청 강원특별자치도춘천교육지원청,강원도 춘천시,신동초 이전 신축공사 관급자재(인조잔디) 구입,신동초 이전 신축공사 관급자재 인조잔디 구입,강원,춘천,춘천,초,초 학교,초,춘천초등학교,신동초
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,충청남도,예산교육지원청,예산여자중학교 유해운동장 교체,예산여자중학교 유해운동장 교체,충남,충청남도예산,충청남도교육청 삽교고등학교,충청남도 예산군,삽교고등학교 인조잔디 운동장 조성공사 관급자재 구매 설치(인조잔디),삽교고등학교 인조잔디 운동장 조성공사 관급자재 구매 설치 인조잔디,충남,예산,예산,운동장 학교,중 학교,고 학교,예산여자중학교,삽교고등학교
104,충청남도,예산교육지원청,예산여자중학교 유해운동장 교체,예산여자중학교 유해운동장 교체,충남,충청남도예산,충청남도교육청 충청남도예산교육지원청,충청남도 예산군,관급자재 구입(신례원초등학교 정구장 돔 증축공사 인조잔디),관급자재 구입 신례원초등학교 정구장 증축공사 인조잔디,충남,예산,예산,학교,중 학교,초 학교,예산여자중학교,신례원초등학교
105,충청북도,청주교육지원청,서경중학교 운동장 보수 및 바닥교체,서경중학교 운동장 보수 및 바닥교체,충북,충청북도청주,충청북도 청주시 공원산림본부,충청북도 청주시 상당구,중하어린이공원 정비용 인조잔디 조달구매,중하어린이공원 정비용 인조잔디 조달구매,충북,청주,청주,중,중 학교,중,서경중학교,
106,충청북도,청주교육지원청,석교초등학교 야구부실내연습장증축,석교초등학교 야구부실내연습장증축,충북,충청북도청주,한국농어촌공사 충북지역본부 청주지사,충청북도 청주시 상당구,낭성면 기초생활거점조성사업 건축토목공사(인조잔디),낭성면 기초생활거점조성사업 건축토목공사 인조잔디,충북,청주,청주,초,초 학교,초,석교초등학교,기초


In [118]:
# 모든 키워드 합 중복 제거
kwd_columns = ['all_kwd', '과업명_키워드1', '납품요구건명_키워드1', '과업명_키워드2', '납품요구건명_키워드2']
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin[kwd_columns].apply(lambda x : (' ').join(x.dropna()),axis=1)
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin['all_kwd_fin'].str.strip()
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin['all_kwd_fin'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
# mapping_merge_df_final_grp_fin.drop(columns=kwd_columns, inplace
# =True)

mapping_merge_df_final_grp_fin = mapping_merge_df_final_grp_fin.drop_duplicates(['시도', '시군구', '과업명', '과업명_re', '시도_re', '시군구_re', '수요기관명', '수요기관지역명',
                                                                                 '납품요구건명', '납품요구건명_re', '수요기관지역명_split1', '수요기관지역명_split2', '시군구_re2', 'all_kwd_fin']).reset_index(drop=True)

In [119]:
# 최종필터링1 - 지역 일치
mapping_merge_df_final_grp_fin_temp1 = mapping_merge_df_final_grp_fin[(mapping_merge_df_final_grp_fin['시군구_re2']!='') & (mapping_merge_df_final_grp_fin['수요기관지역명_split2']!='')]
mapping_merge_df_final_grp_fin_temp1 = mapping_merge_df_final_grp_fin_temp1[mapping_merge_df_final_grp_fin_temp1['시군구_re2']==mapping_merge_df_final_grp_fin_temp1['수요기관지역명_split2']]

In [120]:
# 최종필터링2 - 필수키워드 패턴 포함 & 지역 불일치 제거
mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin[(mapping_merge_df_final_grp_fin['과업명_키워드2']!='')&(mapping_merge_df_final_grp_fin['납품요구건명_키워드2']!='')].reset_index(drop=True)

# 필요키워드 패턴 포함여부 확인
mapping_merge_df_final_grp_fin_temp2['과업명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['과업명_키워드2'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin_temp2[mapping_merge_df_final_grp_fin_temp2.apply(check_inclusion2,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_fin_temp2['과업명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['과업명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

# 지역 불일치 제거
mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin_temp2[mapping_merge_df_final_grp_fin_temp2['시군구_re2']==mapping_merge_df_final_grp_fin_temp2['수요기관지역명_split2']]

In [121]:
# 최종 필터링 데이터셋
mapping_merge_df_final_grp_final = pd.concat([mapping_merge_df_final_grp_fin_temp1,mapping_merge_df_final_grp_fin_temp2]).drop_duplicates().reset_index(drop=True)
mapping_merge_df_final_grp_final.drop(columns = 'all_kwd', inplace=True)

In [127]:
mapping_merge_df_final_grp_fin_temp1[mapping_merge_df_final_grp_fin_temp1['납품요구건명'].str.contains('정목')]

,시도,시군구,과업명,과업명_re,시도_re,시군구_re,수요기관명,수요기관지역명,납품요구건명,납품요구건명_re,수요기관지역명_split1,수요기관지역명_split2,시군구_re2,all_kwd,과업명_키워드1,납품요구건명_키워드1,과업명_키워드2,납품요구건명_키워드2,all_kwd_fin
63,서울특별시,강서양천교육지원청,정목초등학교(운동장 환경 개선),정목초등학교 운동장 환경 개선,서울,서울특별시강서양천,서울특별시강서교육청 서울정목초등학교,서울특별시 양천구,서울정목초 운동장환경개선 인조잔디 구매,서울정목초 운동장환경개선 인조잔디 구매,서울,양천,양천,운동장 초,초 학교,초,정목초등학교,서울정목초,서울정목초 운동장 정목초등학교 초 학교


In [57]:
# mapping_merge_df_final_grp_final.to_csv('C:\py_src\midoproject\data/filtered_mapping_df.csv',encoding='utf-8-sig',index=False)

In [29]:
# 빅쿼리에 적재
save_dataframe_to_bigquery(budget_df_today_final,'budget','budget_df_listup',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_delete,'budget','budget_df_delete',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_new,'budget','budget_df_new',bigquerykey_path)

Data inserted into table budget_df_listup successfully.
Data inserted into table budget_df_delete successfully.
Data inserted into table budget_df_new successfully.
